# Modelo de factorización de matrices: Alternating Least Squares


### Summary

Alternating Least Squares es un modelo de recomendación de factorización de matrices con ratings implícitos. La ventaja que tiene este modelo sobre otros, es que la representación implícita no se representa de manera binaria, sino que permite codificar un nivel de confianza sobre la interacción entre usuarios e ítems. Según el [paper del modelo](http://yifanhu.net/PUB/cf.pdf), la confianza *cui* de que un usuario *u* tenga interés en un ítem *i*, está dada por la siguiente ecuación.

\begin{equation*}
c_{ui} = 1 + α r_{ui}
\end{equation*}

Donde α es una constante (hiperparámetro del modelo), que empíricamente se recomienda setear en 40.

rui son los ratings implícitos que se utilizarán. Se busca que este valor sea mayor cuando exista una mayor señal de interacción del usuario sobre un ítem. Así que viene genial para utilizar como rui la fórmula ya encontrada para pesar los ítems según cantidad de pageviews y la posiciones temporales de estas visitas. Recapitulando, si se toman las posiciones de todas las visitas en una sesión sobre un ítem, dónde la posición 1 es la posición más cercana al la compra, se puede formar un rating con esta fórmula:

\begin{equation*}
r_{ui} = \sum_{pos} \frac{1}{log_{10}(pos + 1)}
\end{equation*}

Lo interesante es que de esta forma se codifica la temporalidad en las sesiones. Otro enfoque interesante fue codificar a las ventas con un rui bastante mayor a los referidos a las visitas, así de esta manera, tendrán mayor peso sobre la función loss.

Otro enfoque adicional fueron codificar atributos de los ítems (dominios, países, términos de títulos y precios), como también los términos usados en las búsquedas, como si fuesen ítems adicionales. También en este caso se utiliza el score que relaciona cantidad de pageviews y la temporalidad, pero con un peso menor que los ítems, cuestión de indicarle al modelo que ponga más foco sobre los ítems. La idea de este enfoque es agregar semántica al entrenamiento, sobre todo para ítems con pocas interacciones; y que se suela recomendar productos de un mismo dominio, siendo esto último muy importante para la métrica de evaluación usada.


### Resultados

Se entrenaron dos modelos, con un conjunto de atributos distintos. En el primer modelo se formó, usando adicionalmente a los ítems: dominios, precios, país y términos de búsquedas. El **ndcg** en test fue **0.3082**.

El segundo modelo se utilizaron, en adición a los ítems, los términos de los títulos de las publicaciones. En este caso el **ndcg** en test fue **0.2981**.

Luego se ensamblaran estos resultados para generar una mejor predicción.

In [1]:
 #import os
#os.environ["OMP_NUM_THREADS"] = "1"
from scipy import sparse
import json
import numpy as np
from collections import Counter
import heapq
import pickle
import implicit

Mapea ítems con dominios, y país del dominio

In [2]:
ITEM_TO_DOMAIN = {}
with open("./data/item_data.jl", "rt") as fd:
    for line in fd:
        data = json.loads(line)
        ITEM_TO_DOMAIN[data["item_id"]] = data["domain_id"]

In [3]:
ITEM_TO_COUNTRY = {}
with open("./data/item_data.jl", "rt") as fd:
    for line in fd:
        data = json.loads(line)
        ITEM_TO_COUNTRY[data["item_id"]] =data["category_id"][2]

Métrica de evaluación

In [4]:
IDCG = np.sum([(1 if i != 1 else 12) / np.log2(1 + i) for i in range(1, 11)])

def dcg(rec, y_item_id, n=10):
    y_domain = ITEM_TO_DOMAIN[y_item_id]
    
    return np.sum([(1 if yhat_item_id != y_item_id else 12) / np.log2(1 + i)\
                   for i, yhat_item_id in enumerate(rec[:n], 1)\
                  if (ITEM_TO_DOMAIN[yhat_item_id] == y_domain)])

## Indexa ítems

Se toman para el modelo ítems con visitas en al menos dos sesiones o que tengan compras en test.

In [5]:
views_counter = Counter()
bought_counter = Counter()

# datos de entrenamiento
with open("./data/train_dataset-train_split.jl", "rt") as fd:
    for line in fd:
        data = json.loads(line)

        item_bought = data["item_bought"]
        items_views = set([event["event_info"] for event in data["user_history"] if event["event_type"] == "view"])
        views_counter.update(items_views)
        bought_counter[item_bought] += 1

# datos de test
with open("./data/train_dataset-test_split.jl", "rt") as fd:
    for line in fd:
        data = json.loads(line)
        item_bought = data["item_bought"]
        items_views = set([event["event_info"] for event in data["user_history"] if event["event_type"] == "view"])
        views_counter.update(items_views)
        
# datos de validación
with open("./data/train_dataset-val_split.jl", "rt") as fd:
    for line in fd:
        try:
            data = json.loads(line)
        except:
            continue
        item_bought = data["item_bought"]
        items_views = set([event["event_info"] for event in data["user_history"] if event["event_type"] == "view"])
        views_counter.update(items_views)

# datos de submission
with open("./data/test_dataset.jl", "rt") as fd:
    for line in fd:
        data = json.loads(line)
        items_views = set([event["event_info"] for event in data["user_history"] if event["event_type"] == "view"])
        views_counter.update(items_views)
        

In [6]:
print(f"Cantidad de artículos en sesiones {len(views_counter): ,d}")

Cantidad de artículos en sesiones  2,098,660


In [7]:
print(f"Cantidad de artículos con más de una sesión {np.sum([v > 1 for v in  views_counter.values() ]): ,d}")

Cantidad de artículos con más de una sesión  536,257


In [8]:
print(f"Cantidad de artículos al menos 3 sesiones {np.sum([v >= 3 for v in  views_counter.values() ]): ,d}")

Cantidad de artículos al menos 3 sesiones  285,976


In [9]:
minsup = 2

items = set([k for k, c in  views_counter.items() if c >= minsup]) | set(bought_counter.keys())

ITEM_TO_IDX = {item_id: idx  for idx, item_id in enumerate(items)}
IDX_TO_ITEM = {idx: item_id   for item_id, idx  in ITEM_TO_IDX.items()}

del items

In [10]:
n_items = len(ITEM_TO_IDX)

In [11]:
n_items

542583

## Indexa Dominios
Se indexan los dominos para incorporar a los datos de entrenamiento.

In [12]:
print(f"Dominios de catálogo: {len(set([domain_id for domain_id in ITEM_TO_DOMAIN.values() if domain_id])):,d}" )

Dominios de catálogo: 7,893


In [13]:
# datos de entrenamiento
domain_counter = Counter()
with open("./data/train_dataset-train_split.jl", "rt") as fd:
    for line in fd:
        data = json.loads(line)

        item_bought = data["item_bought"]
        items_views = set([event["event_info"] for event in data["user_history"] if event["event_type"] == "view"])
        items_views.add(item_bought)
        domains = set([ITEM_TO_DOMAIN[item_id] for item_id in items_views if ITEM_TO_DOMAIN[item_id]])
        
        domain_counter.update(domains)

# datos de test
with open("./data/train_dataset-test_split.jl", "rt") as fd:
    for line in fd:
        data = json.loads(line)

        item_bought = data["item_bought"]
        items_views = set([event["event_info"] for event in data["user_history"] if event["event_type"] == "view"])
        domains = set([ITEM_TO_DOMAIN[item_id] for item_id in items_views if ITEM_TO_DOMAIN[item_id]])
        
        domain_counter.update(domains)

# datos de validacion
with open("./data/train_dataset-val_split.jl", "rt") as fd:
    for line in fd:
        try:
            data = json.loads(line)
        except:
            continue

        item_bought = data["item_bought"]
        items_views = set([event["event_info"] for event in data["user_history"] if event["event_type"] == "view"])
        domains = set([ITEM_TO_DOMAIN[item_id] for item_id in items_views if ITEM_TO_DOMAIN[item_id]])
        
        domain_counter.update(domains)

# datos de submission
with open("./data/test_dataset.jl", "rt") as fd:
    for line in fd:
        try:
            data = json.loads(line)
        except:
            continue


        items_views = set([event["event_info"] for event in data["user_history"] if event["event_type"] == "view"])
        domains = set([ITEM_TO_DOMAIN[item_id] for item_id in items_views if ITEM_TO_DOMAIN[item_id]])
        
        domain_counter.update(domains)

In [14]:
print(f"Dominios en prodoductos con sesiones: {len(domain_counter):,d}")

Dominios en prodoductos con sesiones: 7,893


In [15]:
print(f"Cantidad de categorías con una única sesión {np.sum([v == 1 for v in  domain_counter.values() ]): ,d}")
print(f"Cantidad de categorías con al menos 2 sesiones {np.sum([v >= 2 for v in  domain_counter.values() ]): ,d}")
print(f"Cantidad de categorías con al menos 3 sesiones {np.sum([v >= 3 for v in  domain_counter.values() ]): ,d}")
print(f"Cantidad de categorías con al menos 4 sesiones {np.sum([v >= 4 for v in  domain_counter.values() ]): ,d}")
print(f"Cantidad de categorías con al menos 5 sesiones {np.sum([v >= 5 for v in  domain_counter.values() ]): ,d}")

Cantidad de categorías con una única sesión  527
Cantidad de categorías con al menos 2 sesiones  7,366
Cantidad de categorías con al menos 3 sesiones  6,952
Cantidad de categorías con al menos 4 sesiones  6,651
Cantidad de categorías con al menos 5 sesiones  6,376


In [16]:
minsup = 2

domains = [k for k, c in  domain_counter.most_common() if c >= minsup]

DOMAIN_TO_IDX = {domain_id: idx  for idx, domain_id in enumerate(domains)}
IDX_TO_DOMAIN = {idx: domain_id   for domain_id, idx  in DOMAIN_TO_IDX.items()}

del domains

In [17]:
len(DOMAIN_TO_IDX)

7366

## Atributos Items: Precios
Se definen bins para discretizar los precios y así utilizarlos como atributos adicionales.

In [18]:
PRICE_ITEM = {}
with open("./data/item_data.jl", "rt") as fd:
    for line in fd:
        data = json.loads(line)
        PRICE_ITEM[data["item_id"]] = float(data["price"]) if data["price"] else 0.0

In [19]:
max([price for price in PRICE_ITEM.values() if price >= 1 ])

9999999999.0

In [20]:
PRICES_BINS = [0, 5]
for i in range(1, 10):
    price_break = 10**(i)
    PRICES_BINS.append(price_break)
    if  1 < i <= 4:
        PRICES_BINS.extend([price_break + 10**(i) * j for j in range(1, 9)])
    else:
        PRICES_BINS.append(price_break + 10**(i) * 4 )
PRICES_BINS.append(10000000000)

In [21]:
print(PRICES_BINS)

[0, 5, 10, 50, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000, 20000, 30000, 40000, 50000, 60000, 70000, 80000, 90000, 100000, 500000, 1000000, 5000000, 10000000, 50000000, 100000000, 500000000, 1000000000, 5000000000, 10000000000]


In [22]:
prices_rng_counter = Counter(np.digitize([price for price in PRICE_ITEM.values() if price], PRICES_BINS) - 1)

In [23]:
print(prices_rng_counter.most_common())

[(4, 420631), (3, 361576), (2, 358583), (5, 192863), (13, 131698), (6, 115995), (7, 77586), (8, 58147), (14, 48000), (9, 46190), (10, 36773), (11, 30750), (12, 28936), (22, 25489), (15, 25097), (31, 22827), (16, 15405), (23, 15391), (24, 12032), (17, 10601), (1, 8921), (25, 8142), (18, 7566), (19, 6451), (26, 5801), (20, 5052), (21, 4617), (27, 4473), (28, 3620), (29, 2880), (32, 2861), (30, 2659), (0, 1942), (33, 1658), (35, 256), (34, 248), (37, 62), (39, 60), (38, 52), (36, 39), (40, 8)]


## Atributos Items: Títulos de publicaciones

Se seleccionan términos en los títulos de las publicaciones. Para la selección de los términos se utilizaron reglas de asociación de tamaño 2 entre los términos (antecedente) y los dominios (consecuente). De esta forma se busca elegir términos con mayor contenido semántico.

In [24]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
import unidecode
import pandas as pd

In [25]:
def text_norm(text):
    return unidecode.unidecode(text).lower()

stop_words = stopwords.words('portuguese') + stopwords.words('spanish')
stop_words =  set([text_norm(w) for w in stop_words])

tokenizer = RegexpTokenizer(r'\w{2,}')
def tokenize(text):
    return [w for w in tokenizer.tokenize(text_norm(text)) if w not in stop_words]

In [26]:
# True para levantar datos precalculados
LOAD = True

In [27]:
if not LOAD:
    # cuenta frecuencias de dominios o tokens
    domain_count = Counter()
    word_count = Counter()
    N = 0
    with open("./data/item_data.jl", "rt") as fd:
        for line in fd:
            data = json.loads(line)
            if data["domain_id"]:
                domain_count[data["domain_id"]] += 1
                word_count.update(set(tokenize(data["title"])))
                N += 1
    # selecciona dominios o tokens por soporte mínimo
    min_sup = 5
    domain_count = Counter({k: c for k, c in domain_count.items() if c >= min_sup})
    word_count = Counter({k: c for k, c in word_count.items() if c >= min_sup})
    
    # Itemsets de tamaño 2
    domain_word_count = Counter()
    with open("./data/item_data.jl", "rt") as fd:
        for line in fd:
            data = json.loads(line)
            if data["domain_id"] in domain_count:
                domain_word_count.update([(data["domain_id"], word)\
                                         for word in set(tokenize(data["title"])) if word in word_count])
            
    # calcula métricas de reglas de asociación
    df_asso = pd.DataFrame([[d, w, c, (c / word_count[w]), (c * N / (word_count[w] * domain_count[d]) if domain_count[d] else 0)]\
                        for (d, w), c  in domain_word_count.items()],
                        columns=["domain_id", "word", "sup_count", "conf", "lift"])

    # selecciona asociaciones por métricas
    df_asso = df_asso[(df_asso.lift > 1.1) & (df_asso.conf >= 0.2) & (df_asso.sup_count >= min_sup) ]

Asociaciones más fuertes

In [38]:
if not LOAD:
    from IPython.display import display
    display(df_asso.sort_values("lift", ascending=False).head(10))

,domain_id,word,sup_count,conf,lift
127266,MLM-CROWBARS,barreta,5,0.833333,350237.666667
1352389,MLB-TENNIS_AND_SQUASH_RACKET_VIBRATION_DAMPENERS,antivibrador,6,1.000000,350237.666667
374873,MLM-GINS,ginebra,5,0.714286,300203.714286
998404,MLB-ROCK_CRUSHERS,britador,6,0.857143,300203.714286
126847,MLM-PICKGUARDS,pickguard,5,0.714286,300203.714286
173867,MLM-IGNITION_KNOCK_SENSORS,detonacion,5,0.833333,291864.722222
1187904,MLB-ANALOG_RAIN_GAUGES,pluviometro,5,0.625000,262678.250000
139737,MLM-TAMBOURINES,pandero,6,1.000000,233491.777778
598697,MLB-KAZOOS,kazoo,5,0.555556,233491.777778
178202,MLM-EDGE_BANDING_MACHINES,enchapadora,7,0.777778,233491.777778


Indexa los términos seleccionados

In [28]:
if not LOAD:
    # palabras seleccionadas
    items_tokens = df_asso.word.unique()

    # indexa palabras/tokens
    ITEM_TOKEN_TO_IDX = {w: idx for idx, w in enumerate(items_tokens)}
    IDX_TO_ITEM_TOKEN = {idx: feat for feat, idx  in ITEM_TOKEN_TO_IDX.items()}

    # calcula document frecuency del token
    DF_ITEM_TOKEN = {ITEM_TOKEN_TO_IDX[word]: word_count[word] for word in items_tokens}

    ITEM_TO_TOKENS_IDX = {}
    with open("./data/item_data.jl", "rt") as fd:
        for line in fd:
            data = json.loads(line)        
            ITEM_TO_TOKENS_IDX[data["item_id"]] =\
                [ITEM_TOKEN_TO_IDX[w] for w in set(tokenize(data["title"])) if w in ITEM_TOKEN_TO_IDX]
            

Guarda o levanta índices

In [29]:
if LOAD == False:
    var = {
        "ITEM_TO_TOKENS_IDX": ITEM_TO_TOKENS_IDX,
        "ITEM_TOKEN_TO_IDX": ITEM_TOKEN_TO_IDX,
        "IDX_TO_ITEM_TOKEN": IDX_TO_ITEM_TOKEN,
        "DF_ITEM_TOKEN": DF_ITEM_TOKEN,
    }
    
    with open("data/models/items_title_tokens.pkl", "wb") as fd:
        pickle.dump(var, fd, protocol=pickle.HIGHEST_PROTOCOL)
else:
    with open("data/models/items_title_tokens.pkl", "rb") as fd:
        var = pickle.load(fd)
        ITEM_TO_TOKENS_IDX = var["ITEM_TO_TOKENS_IDX"] 
        ITEM_TOKEN_TO_IDX = var["ITEM_TOKEN_TO_IDX"]
        IDX_TO_ITEM_TOKEN = var["IDX_TO_ITEM_TOKEN"]
        DF_ITEM_TOKEN = var["DF_ITEM_TOKEN"]


## Indexa búsquedas
Se indexan términos (o tokens) de utilizados en búsquedas.

In [30]:
search_counter = Counter()
n_users = 0
# datos de entrenamiento
with open("./data/train_dataset-train_split.jl", "rt") as fd:
    for line in fd:
        data = json.loads(line)
        searchs = set([event["event_info"].upper().strip() for event in data["user_history"] if event["event_type"] == "search"])
        search_counter.update(searchs)
        n_users += 1
        
# datos de test
with open("./data/train_dataset-test_split.jl", "rt") as fd:
    for line in fd:
        data = json.loads(line)
        searchs = set([event["event_info"].upper().strip() for event in data["user_history"] if event["event_type"] == "search"])
        search_counter.update(searchs)
        n_users += 1
        
# datos de val
with open("./data/train_dataset-val_split.jl", "rt") as fd:
    for line in fd:
        try:
            data = json.loads(line)
        except:
            continue
        searchs = set([event["event_info"].upper().strip() for event in data["user_history"] if event["event_type"] == "search"])
        search_counter.update(searchs)
        n_users += 1
        
# datos de submission
with open("./data/test_dataset.jl", "rt") as fd:
    for line in fd:
        try:
            data = json.loads(line)
        except:
            continue
        searchs = set([event["event_info"].upper().strip() for event in data["user_history"] if event["event_type"] == "search"])
        search_counter.update(searchs)
        n_users += 1

In [31]:
print(f"Cantidad de búsquedas únicas en sesiones {len(search_counter): ,d}")

Cantidad de búsquedas únicas en sesiones  1,157,186


In [32]:
print(f"Cantidad de búsquedas con más de una sesión {np.sum([v > 1 for v in  search_counter.values() ]): ,d}")

Cantidad de búsquedas con más de una sesión  158,894


In [33]:
print(f"Cantidad de búsquedas con al menos 3 sesiones {np.sum([v >= 3 for v in  search_counter.values() ]): ,d}")

Cantidad de búsquedas con al menos 3 sesiones  80,584


Cuenta tokens

In [34]:
token_counter = Counter()
for text in search_counter.keys():
    token_counter.update(set(text.split()))

In [35]:
print(f"Cantidad de tokens únicas en búsquedas {len(token_counter): ,d}")

Cantidad de tokens únicas en búsquedas  143,370


In [36]:
print(f"Cantidad de tokens en más de una búsqueda {np.sum([v > 1 for v in  token_counter.values() ]): ,d}")

Cantidad de tokens en más de una búsqueda  74,968


In [37]:
print(f"Cantidad de tokens en al menos 3 búsquedas {np.sum([v >= 3 for v in  token_counter.values() ]): ,d}")

Cantidad de tokens en al menos 3 búsquedas  54,712


In [38]:
print(f"Cantidad de tokens en al menos 5 búsquedas {np.sum([v >= 5 for v in  token_counter.values() ]): ,d}")

Cantidad de tokens en al menos 5 búsquedas  38,245


Selecciona tokens

In [39]:
minsup = 5

searchs = set([k for k, c in  search_counter.items() if c >= minsup and len(k) >= 3])

minsup = 5
tokens = set([k for k, c in  token_counter.items() if c >= minsup and len(k) >= 3])


In [40]:
tokens_search_pop = set([ token for text in searchs for token in text.split() if len(token) >=3])

In [41]:
len(tokens), len(tokens_search_pop), len(tokens | tokens_search_pop) 

(37152, 12263, 37698)

In [42]:
tokens = tokens | tokens_search_pop

In [43]:
SEARCH_TOKEN_TO_IDX = {feat: idx  for idx, feat in enumerate(tokens)}
IDX_TO_SEARCH_TOKEN = {idx: feat for feat, idx  in SEARCH_TOKEN_TO_IDX.items()}
DF_SEARCH_TOKEN = {feat: token_counter[feat]  for feat in tokens}

# Matriz utilidad

La matriz de utilidad relaciona a los usuarios o sesiones (filas), con los ítems visitados o comprados (columnas). Adicionalmente se codifican como si fuesen otros ítems (columnas), los atributos derivados de los ítems de la sesión (categoría, país, precios, tokens de título), cómo también los términos usados en las búsquedas.

Esta representación de matriz es implícita, pero en vez de utilizar una representación binaria, se utiliza un score para pesar la confianza que se tiene en la interacción user/ítem. Este score tiene en cuenta la cantidad de visitas en la sesión sobre el ítem, el órden temporal de esas visitas, y si el ítem fue finalmente comprado (esto último para datos de test)

In [44]:
print(f"Items: {len(ITEM_TO_IDX):,d} - "
      f"Dominios: {len(DOMAIN_TO_IDX):,d} - "
      f"Paises: 2 - "
      f"Precios: {len(PRICES_BINS)} - "
      f"Title tokens: {len(ITEM_TOKEN_TO_IDX):,d} - "
      f"Search Tokens: {len(SEARCH_TOKEN_TO_IDX):,d}")

Items: 542,583 - Dominios: 7,366 - Paises: 2 - Precios: 42 - Title tokens: 48,804 - Search Tokens: 37,698


In [45]:
def get_interactions(user_idx, items, item_weight):
    """
    Genera coordenadas de matriz rala para representar las interacciones.
    
    @param user_idx: Índice de fila
    @param items: Listado de ítems visitados o comprados (en formato de id de catálogo)
    @param item_weight: Pesos o scores del ítem en la sesión.
    
    return Tulpas de dimensión 5 con las siguientes valores:
     - Posición 0: Coordenadas y valores de score de iteracciones con ítems. (Listado de tuplas de tamaño 3: (user_idx, item_idx, score))
     - Posición 1: Coordenadas de iteracciones con dominios. (mismo formato que ítems)
     - Posición 2: Coordenadas de interacciones con países (mismo formato que ítems)
     - Posición 3: Coordenadas de intenteracciones con bins de precios. (mismo formato que ítems)
     - Posición 4: Coordenadas de intenteracciones tokens de títulos. (mismo formato que ítems)
    """
    global ITEM_TO_IDX, ITEM_TO_DOMAIN, PRICE_ITEM, DOMAIN_TO_IDX, ITEM_TO_COUNTRY, PRICES_BINS,\
           DF_ITEM_TOKEN, ITEM_TO_TOKENS_IDX
    
    # items
    inte_items = [(user_idx, ITEM_TO_IDX[item_id], weight)\
                  for item_id, weight in zip(items, item_weight) if item_id in ITEM_TO_IDX]

    # dominios
    domain_weight = {}
    for item_id, weight in zip(items, item_weight):
        domain_id = ITEM_TO_DOMAIN[item_id] 
        if domain_id in DOMAIN_TO_IDX:
            domain_idx = DOMAIN_TO_IDX[domain_id] 
            domain_weight[domain_idx] = domain_weight.get(domain_idx, 0) + weight
    inte_domains = [(user_idx, domain_ix, weight / len(items) ) for domain_ix, weight in domain_weight.items()]

    # paises
    inte_counties = [(user_idx, 0 if c == "B" else 1, 1)  for c in set([ITEM_TO_COUNTRY[item_id] for item_id in items])]

    # precios
    price_weight = {}
    for item_id, weight in zip(items, item_weight):
        price = PRICE_ITEM[item_id] 
        if price:
            price_idx = np.digitize(price, PRICES_BINS) - 1
            price_weight[price_idx] = price_weight.get(price_idx, 0) + weight
    inte_prices = [(user_idx, price_idx, weight  / len(items) ) for price_idx, weight in price_weight.items()]
    
    # title
    token_weight = {}
    for item_id, weight in zip(items, item_weight):
        tokens_idx = ITEM_TO_TOKENS_IDX[item_id] 
        for token_idx in tokens_idx:
            token_weight[token_idx] = token_weight.get(token_idx, 0) + weight
    inte_tokens = [(user_idx, token_idx, tf/ DF_ITEM_TOKEN[token_idx]) for token_idx, tf in token_weight.items()]
            
    return inte_items, inte_domains, inte_counties, inte_prices, inte_tokens


In [46]:
def get_search_interactions(user_idx, texts, weights):
    """
    Genera coordenadas de matriz rala para representar las interacciones de los términos de busquedas.
    
    @param user_idx: Índice de fila
    @param texts: Texto de búsqueda
    @param weights: Pesos o scores según posición en la sesión
    
    return  Coordenadas y valores de score de iteracciones con términos. (Listado de tuplas de tamaño 3: (user_idx, token, score))
    """
    global SEARCH_TOKEN_TO_IDX, DF_SEARCH_TOKEN
    
    token_tf = {}
    for text, w in zip(texts, weights):
            
        for token in set(text.split()):
            if token in SEARCH_TOKEN_TO_IDX:
                token_tf[token] = token_tf.get(token, 0) + w
    inte_tokens = [(user_idx, SEARCH_TOKEN_TO_IDX[token], tf/ DF_SEARCH_TOKEN[token]) for token, tf in token_tf.items()]
    
    return inte_tokens
    

In [47]:
def read_iteractions(path, user_idx = 0, is_train=False):
    """
    Genera índices y valores de matriz.
    """
    interaction_items = []
    interactions_domains = []
    interactions_country = []
    interactions_prices = []
    interactions_item_token = []
    interactions_search_token = []
    y = []
    BOUGHT_WEIGHT= 30

    with open(path, "rt") as fd:
        for line in fd:
            try:
                data = json.loads(line)
            except:
                continue

            item_bought = data.get("item_bought")
            y.append(item_bought)
            
            events = data["user_history"][::-1]
            items_views = [event["event_info"] for event in events if event["event_type"] == "view"]
            searchs = [event["event_info"].upper().strip() for event in events if event["event_type"] == "search"]

            # Ranking de items visitados
            items_pv_count = {}
            for pos, item_view in enumerate(items_views, 1):
                items_pv_count[item_view] = items_pv_count.get(item_view, 0) + 1 / np.log10(pos + 1)
            if is_train:
                items_pv_count[item_bought] = BOUGHT_WEIGHT
            items, item_weight = zip(*items_pv_count.items()) if items_pv_count else ([], [])

            # ranking de busquedas
            search_weights = {}
            for pos, text in enumerate(searchs, 1):
                search_weights[text] = search_weights.get(text, 0) + 1 / np.log10(pos + 1)
            texts, search_weights = zip(*search_weights.items()) if search_weights else ([], [])

            # índices items/atributos
            inte_i, inte_d, inte_c, inte_p, inte_itkn = get_interactions(user_idx, items, item_weight)
            interactions_items.extend(inte_i)
            interactions_domains.extend(inte_d)
            interactions_country.extend(inte_c)
            interactions_prices.extend(inte_p)
            interactions_item_token.extend(inte_itkn)

            # pindices búquedas
            inte_stkn  = get_search_interactions(user_idx, texts, search_weights)
            interactions_search_token.extend(inte_stkn)
            user_idx += 1
    return (user_idx, y, interactions_items, interactions_domains, interactions_country,
            interactions_prices, interactions_item_token, interactions_search_token)

Obtiene iteracciones en fromato de lista de tuplas (row, column, value)

In [48]:
interactions_items = []
interactions_domains = []
interactions_country = []
interactions_prices = []
interactions_item_token = []
interactions_search_token = []

# train
user_idx = 0 
iteractions = read_iteractions("./data/train_dataset-train_split.jl", user_idx, True)
user_idx = iteractions[0]
interactions_items.extend(iteractions[2])
interactions_domains.extend(iteractions[3])
interactions_country.extend(iteractions[4])
interactions_prices.extend(iteractions[5])
interactions_item_token.extend(iteractions[6])
interactions_search_token.extend(iteractions[7])

# test
test_ini_idx = user_idx
iteractions = read_iteractions("./data/train_dataset-test_split.jl", user_idx, False)
user_idx = iteractions[0]
y_test = iteractions[1]
interactions_items.extend(iteractions[2])
interactions_domains.extend(iteractions[3])
interactions_country.extend(iteractions[4])
interactions_prices.extend(iteractions[5])
interactions_item_token.extend(iteractions[6])
interactions_search_token.extend(iteractions[7])


# val
val_ini_idx = user_idx
iteractions = read_iteractions("./data/train_dataset-val_split.jl", user_idx, False)
user_idx = iteractions[0]
y_val = iteractions[1]
interactions_items.extend(iteractions[2])
interactions_domains.extend(iteractions[3])
interactions_country.extend(iteractions[4])
interactions_prices.extend(iteractions[5])
interactions_item_token.extend(iteractions[6])
interactions_search_token.extend(iteractions[7])

# datos de submission
sub_ini_idx = user_idx
iteractions = read_iteractions("./data/test_dataset.jl", user_idx, False)
user_idx = iteractions[0]
interactions_items.extend(iteractions[2])
interactions_domains.extend(iteractions[3])
interactions_country.extend(iteractions[4])
interactions_prices.extend(iteractions[5])
interactions_item_token.extend(iteractions[6])
interactions_search_token.extend(iteractions[7])


del iteractions
n_users = user_idx

In [49]:
n_users = user_idx
row, col, data  = zip(*interactions_items)
ui_matrix = sparse.coo_matrix((data, (row, col)), shape = (n_users, len(ITEM_TO_IDX)) ).tocsr()
ui_matrix

<590231x542583 sparse matrix of type '<class 'numpy.float64'>'
	with 3508641 stored elements in Compressed Sparse Row format>

In [50]:
n_users = user_idx
row, col, data  = zip(*interactions_domains)
ud_matrix = sparse.coo_matrix((data, (row, col)), shape = (n_users, len(DOMAIN_TO_IDX)) ).tocsr()
ud_matrix

<590231x7366 sparse matrix of type '<class 'numpy.float64'>'
	with 2078509 stored elements in Compressed Sparse Row format>

In [51]:
n_users = user_idx
row, col, data  = zip(*interactions_country)
uc_matrix = sparse.coo_matrix((data, (row, col)), shape = (n_users, 2) ).tocsr()
uc_matrix

<590231x2 sparse matrix of type '<class 'numpy.int64'>'
	with 576281 stored elements in Compressed Sparse Row format>

In [52]:
n_users = user_idx
row, col, data  = zip(*interactions_prices)
uprices_matrix = sparse.coo_matrix((data, (row, col)) ).tocsr()
uprices_matrix

<590231x41 sparse matrix of type '<class 'numpy.float64'>'
	with 2058834 stored elements in Compressed Sparse Row format>

In [53]:
n_users = user_idx
row, col, data  = zip(*interactions_item_token)
uitoken_matrix = sparse.coo_matrix((data, (row, col))).tocsr()
uitoken_matrix

<590231x48804 sparse matrix of type '<class 'numpy.float64'>'
	with 10221767 stored elements in Compressed Sparse Row format>

In [54]:
n_users = user_idx
row, col, data  = zip(*interactions_search_token)
ustoken_matrix = sparse.coo_matrix((data, (row, col))).tocsr()
ustoken_matrix

<590231x37698 sparse matrix of type '<class 'numpy.float64'>'
	with 3740406 stored elements in Compressed Sparse Row format>

Valida que todas las columnas tengan valores

In [55]:
(ui_matrix.sum(axis=0) == 0).sum()

0

In [56]:
(ud_matrix.sum(axis=0) == 0).sum()

0

In [57]:
(uc_matrix.sum(axis=0) == 0).sum()

0

In [58]:
(uprices_matrix.sum(axis=0) == 0).sum()

0

In [59]:
(uitoken_matrix.sum(axis=0) == 0).sum()

0

In [60]:
(ustoken_matrix.sum(axis=0) == 0).sum()

0

## Matrices en pickle

In [61]:
LOAD = True

In [62]:
if LOAD == False:
    var = {
        # matrices
        "ui_matrix": ui_matrix,
        "ud_matrix": ud_matrix,
        "uc_matrix": uc_matrix,
        "uprices_matrix": uprices_matrix,
        "uitoken_matrix": uitoken_matrix,
        "ustoken_matrix": ustoken_matrix,
        
        # índices
        "ITEM_TO_IDX": ITEM_TO_IDX,
        "DOMAIN_TO_IDX": DOMAIN_TO_IDX,
        "IDX_TO_ITEM": IDX_TO_ITEM,
        "IDX_TO_DOMAIN":IDX_TO_DOMAIN,

        # índices de conjuntos de test y etiquetas
        "test_ini_idx": test_ini_idx,
        "y_test": y_test,
        "val_ini_idx": val_ini_idx,
        "y_val": y_val,
        "sub_ini_idx": sub_ini_idx,
    }
    
    with open("data/models/implicit_matrix_variables.pkl", "wb") as fd:
        pickle.dump(var, fd, protocol=pickle.HIGHEST_PROTOCOL)
else:
    with open("data/models/implicit_matrix_variables.pkl", "rb") as fd:
        var = pickle.load(fd)
        ui_matrix = var["ui_matrix"]
        ud_matrix = var["ud_matrix"]
        uc_matrix = var["uc_matrix"]
        uprices_matrix = var["uprices_matrix"]
        uitoken_matrix = var["uitoken_matrix"]
        ustoken_matrix = var["ustoken_matrix"]
        
        DOMAIN_TO_IDX = var["DOMAIN_TO_IDX"]
        IDX_TO_DOMAIN = var["IDX_TO_DOMAIN"]
        
        
        ITEM_TO_IDX = var["ITEM_TO_IDX"]
        DOMAIN_TO_IDX =  var["DOMAIN_TO_IDX"]
        IDX_TO_ITEM = var["IDX_TO_ITEM"]
        IDX_TO_DOMAIN = var["IDX_TO_DOMAIN"]
        
        test_ini_idx = var["test_ini_idx"]
        y_test = var["y_test"]
        val_ini_idx = var["val_ini_idx"]
        y_val = var["y_val"]
        sub_ini_idx = var["sub_ini_idx"]
    
    

# Entrenamiento de modelos

Se entrenaron dos modelos con distintos conjuntos de atributos:

modelo 1: ítems + dominios + países + precios + tokens de busqueda

modelo 2: ítems + tokens de títulos

## Modelo 1

In [4]:
# obtiene cantidad de ítems a recomendar
n_items = uitems_matrix.shape[1]

In [ ]:
# normaliza matriz de precios (por filas)
up_row_sum = uprices_matrix.sum(axis=1).A.ravel()
c = sparse.diags(1/np.where(up_row_sum > 0, up_row_sum, 1 ))
uprices_matrix = c.dot(uprices_matrix)

In [ ]:
ui_matrix_model1 = sparse.hstack([uitems_matrix, uc_matrix, ud_matrix * 0.5, uprices_matrix , ustoken_matrix ])

Después de una selección de hiperparámetros se entrena este modelo formamdo embebidos de dimensión 1024 (hasta donde dió la memoria, ya que con más resultó mejor)

In [ ]:
model1 = implicit.als.AlternatingLeastSquares(factors=1024,
                                              regularization=0.4, iterations=5, 
                                              num_threads=4, calculate_training_loss=False, 
                                              random_state=123)

alpha_val = 40
data_conf = (ui_matrix_model1.T * alpha_val)
model1.fit(data_conf)

Evaluación de modelo en test. (guarda las primeras 50 recomendaciones para ensamble)

In [ ]:
# evaluación en test
n_recs = 0
sum_dcg = 0

# las recomendacione sólo se hacen sobre productos (no queremos ni atributos, ní búsquedas)
all_items_idx = list(range(n_items, ui_matrix_model1.shape[1]))

# índice de fila inicial de conjunto de test
n_train = test_ini_idx

test_reco_scores = []
for offset, y_item_id in enumerate(y_test):
    # scores
    recommended = model1.recommend(n_train + offset, ui_matrix_model1,
                                  filter_already_liked_items=False,
                                   N=50,
                                   filter_items=all_items_idx,
                                   recalculate_user=False)
    test_reco_scores.append([(IDX_TO_ITEM[item_idx], score) for item_idx, score in recommended])
    rec = [IDX_TO_ITEM[item_idx] for item_idx, score in recommended][:10]
    
    # evaluation
    sum_dcg += dcg(rec, y_item_id)    
    n_recs += 1
        
print(f"NDCG: {sum_dcg / (IDCG * n_recs): .4f} - {n_recs} recomendaciones")

Scores de validación

In [ ]:
val_reco_scores = []
for offset, y_item_id in enumerate(y_val):
    # scores
    recommended = model1.recommend(val_ini_idx + offset, ui_matrix_model1,
                                  filter_already_liked_items=False, N=50,  filter_items=all_items_idx, recalculate_user=False)
    val_reco_scores.append([(IDX_TO_ITEM[item_idx], score) for item_idx, score in recommended])

Guarda predicciones para armar ensamble de modelos

In [ ]:
var_rec = {
    "test_reco_scores": test_reco_scores,
    "val_reco_scores": val_reco_scores,
}

with open("data/models/implicit_test_reco_scores_model1.pkl", "wb") as fd:
    pickle.dump(var_rec, fd, protocol=pickle.HIGHEST_PROTOCOL)

## Modelo 2

In [ ]:
# obtiene cantidad de ítems a recomendar
n_items = ui_matrix.shape[1]

In [ ]:
# Selecciona términos de búsquedas o títulos con al menos 5 sesiones
mask = np.asarray(uitoken_matrix.astype(bool).sum(axis=0) >= 5).squeeze()
uitoken_matrix = uitoken_matrix[:, mask].tocsr()

mask = np.asarray(ustoken_matrix.astype(bool).sum(axis=0) >= 5).squeeze()
ustoken_matrix = ustoken_matrix[:, mask].tocsr()

In [5]:
ui_matrix_model2 = sparse.hstack([uitems_matrix, uitoken_matrix, ustoken_matrix])

En este caso el recurso escaso de la memoria llegó a cubrir embebidos con 750 dimensiones. Se entrena:

In [ ]:
model2 = implicit.als.AlternatingLeastSquares(factors=750,
                                              regularization=0.4, iterations=5, 
                                              num_threads=4, calculate_training_loss=False, 
                                              random_state=123)

alpha_val = 40
data_conf = (ui_matrix_model2.T * alpha_val)
model2.fit(data_conf)

Evaluación en test:

In [6]:
# evaluación en test
n_recs = 0
sum_dcg = 0

# las recomendacione sólo se hacen sobre productos (no queremos ni atributos, ní búsquedas)
all_items_idx = list(range(n_items, ui_matrix_model2.shape[1]))

# índice de fila inicial de conjunto de test
n_train = test_ini_idx

test_reco_scores = []
for offset, y_item_id in enumerate(y_test):
    # scores
    recommended = model2.recommend(n_train + offset, ui_matrix_model2,
                                  filter_already_liked_items=False,
                                   N=50,
                                   filter_items=all_items_idx,
                                   recalculate_user=False)
    test_reco_scores.append([(IDX_TO_ITEM[item_idx], score) for item_idx, score in recommended])
    rec = [IDX_TO_ITEM[item_idx] for item_idx, score in recommended][:10]
    
    # evaluation
    sum_dcg += dcg(rec, y_item_id)    
    n_recs += 1
        
print(f"NDCG: {sum_dcg / (IDCG * n_recs): .4f} - {n_recs} recomendaciones")

Guarda scores en test y validación

In [ ]:
val_reco_scores = []
for offset, y_item_id in enumerate(y_val):
    # scores
    recommended = model2.recommend(val_ini_idx + offset, ui_matrix_model2,
                                  filter_already_liked_items=False, N=50,  filter_items=all_items_idx, recalculate_user=False)
    val_reco_scores.append([(IDX_TO_ITEM[item_idx], score) for item_idx, score in recommended])

In [ ]:
var_rec = {
    "test_reco_scores": test_reco_scores,
    "val_reco_scores": val_reco_scores,
}

with open("data/models/implicit_test_reco_scores_model2.pkl", "wb") as fd:
    pickle.dump(var_rec, fd, protocol=pickle.HIGHEST_PROTOCOL)